In [1]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [2]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types

Cloning into 'pb_gee_tools'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 341 (delta 33), reused 35 (delta 13), pack-reused 271 (from 1)
Receiving objects: 100% (341/341), 563.75 KiB | 11.05 MiB/s, done.
Resolving deltas: 100% (181/181), done.
Processing ./pb_gee_tools
  Preparing metadata (setup.py) ... done
  Created wheel for pb_gee_tools: filename=pb_gee_tools-0.3.0-py3-none-any.whl size=17966 sha256=68ff9fde1a7a9659334a694ede94a0394e8fe6a673d30c33989b699becc09444
  Stored in directory: /tmp/pip-ephem-wheel-cache-9gekltia/wheels/e8/da/17/69dc01c6cbd07adb00923d8c1dd9a22fb31e96a17b74c93812
Successfully built pb_gee_tools


In [30]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [4]:
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
# The file path on google drive for ROI output vector file.
vec_cls_roi_file = "/content/drive/MyDrive/mangrove_chng_cls/n0940e0190_tile.geojson"

# Start and End date
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)

# Output no data value
no_data_val = 0.0

In [7]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_cls_roi_gdf = geopandas.read_file(vec_cls_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_cls_roi_bbox = vec_cls_roi_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_cls_west = gp_cls_roi_bbox[0]
roi_cls_east = gp_cls_roi_bbox[2]
roi_cls_north = gp_cls_roi_bbox[3]
roi_cls_south = gp_cls_roi_bbox[1]
cls_tile_aoi = ee.Geometry.BBox(roi_cls_west, roi_cls_south, roi_cls_east, roi_cls_north)

cls_cnt_pt_x = roi_cls_west + (roi_cls_east - roi_cls_west) / 2
cls_cnt_pt_y = roi_cls_south + (roi_cls_north - roi_cls_south) / 2
cls_cnt_pt = ee.Geometry.Point([cls_cnt_pt_x, cls_cnt_pt_y])

In [8]:
# Load the Sentinel-2 imagery.
s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
    aoi=cls_tile_aoi,
    start_date=start_date,
    end_date=end_date,
    cloud_thres = 70,
)

# Create a median composite clipped to the ROI.
s2_med_image = s2_img_col.median().clip(cls_tile_aoi).unmask(no_data_val)

In [9]:
map_obj = geemap.Map()

# Define a centre point for the map to start at.
cls_cnt_pt_x = 93.89789
cls_cnt_pt_y = 19.30903
cls_cnt_pt = ee.Geometry.Point([cls_cnt_pt_x, cls_cnt_pt_y])

# Specify the centre point and scale of the map
map_obj.centerObject(cls_cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {"min": [150, 100, 187], "max": [4136, 4000, 2712], "bands": ["B8", "B11", "B4"]}

# Add the Sentinel-2 Composite
map_obj.addLayer(s2_med_image, vis_params, "Sentinel-2")

map_obj

Map(center=[19.30903, 93.89789], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [10]:
# Get the list of image bands.
img_bands = s2_med_image.bandNames()
img_bands

In [31]:
# Load the classifier from asset
cls_mdl_asset_id = f'projects/{ee_prj_name}/assets/mng_test_rf_cls'
rf_cls_mdl = ee.Classifier.load(cls_mdl_asset_id)


In [32]:
# Apply the Classification.
cls_rslts_img = s2_med_image.classify(rf_cls_mdl)

In [33]:
map_obj = geemap.Map()

map_obj.centerObject(cls_cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
s2_vis_params = {"min": [150, 100, 187], "max": [4136, 4000, 2712], "bands": ["B8", "B11", "B4"]}

# Add the Sentinel-2 Composite
map_obj.addLayer(s2_med_image, s2_vis_params, "Sentinel-2")

# Specify the visualisation options.
cls_vis_params = {'min': 1, 'max': 3, 'palette': ['green', 'blue', 'grey']}

# Add the Sentinel-2 Classification
map_obj.addLayer(cls_rslts_img, cls_vis_params, "Class")

map_obj

EEException: Classifier asset 'projects/ee-pb-dev/assets/mng_test_rf_cls' not found (does not exist or caller does not have access).

In [ ]:
out_cls_img_file = "n0940e0190_asset_mdl_cls"
out_gdrive_dir = "mangrove_chng_cls"

# Export the classification image to your Google Drive
task = ee.batch.Export.image.toDrive(
        image=cls_rslts_img,
        description=out_cls_img_file,
        folder=out_gdrive_dir,
        scale=10,
        region=cls_tile_aoi,
        fileFormat="GeoTIFF",
        formatOptions={"cloudOptimized": True, 'noData': 0},
)
task.start()